# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [ ]:
import pandas as pd
import numpy as np
from typing import Tuple, List

def cargar_datos() -> pd.DataFrame:
    columnas = ['fecha', 'producto', 'precio', 'cantidad']
    anchos = [10, 30, 10, 5]
    df = pd.read_fwf('datos.dat', widths=anchos, names=columnas)
    df['fecha'] = pd.to_datetime(df['fecha'])
    df[['precio', 'cantidad']] = df[['precio', 'cantidad']].apply(pd.to_numeric)
    return df

datos = cargar_datos()
print(datos.head())

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [ ]:
def calcular_totales(df: pd.DataFrame) -> Tuple[float, int]:
    importe_total = (df['precio'] * df['cantidad']).sum()
    cantidad_total = df['cantidad'].sum()
    return importe_total, cantidad_total

importe, cantidad = calcular_totales(datos)
print(f"Las ventas fueron de ${importe:.2f} en {cantidad} unidades")

### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [ ]:
def unidades_vendidas(df: pd.DataFrame) -> pd.Series:
    return df.groupby('producto')['cantidad'].sum().sort_values(ascending=False)

ventas = unidades_vendidas(datos)
print(ventas)



###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [ ]:
def precio_promedio(df: pd.DataFrame) -> pd.Series:
    return df.groupby('producto')['precio'].mean().round(2)

precios = precio_promedio(datos)
print(precios)

### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [ ]:
def ranking_productos(df: pd.DataFrame, n: int = 3) -> pd.Series:
    return unidades_vendidas(df).head(n)

ranking = ranking_productos(datos)
print(ranking)

### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [ ]:
def ventas_por_mes(df: pd.DataFrame) -> pd.DataFrame:
    df['mes'] = df['fecha'].dt.to_period('M')
    return df.pivot_table(values='cantidad', index='mes', columns='producto', aggfunc='sum', fill_value=0)

ventas_mensuales = ventas_por_mes(datos)
print(ventas_mensuales)

### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [ ]:
def resumen_ventas(df: pd.DataFrame) -> pd.DataFrame:
    resumen = df.groupby('producto').agg({
        'precio': ['mean', lambda x: (x * df.loc[x.index, 'cantidad']).sum()],
        'cantidad': 'sum'
    })
    resumen.columns = ['precio_promedio', 'importe_total', 'unidades_vendidas']
    resumen['precio_promedio'] = resumen['importe_total'] / resumen['unidades_vendidas']
    return resumen.sort_index()

resumen = resumen_ventas(datos)
print(resumen)

## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [ ]:
def cuadratic_function(x: np.ndarray, coefs: np.ndarray) -> np.ndarray:
    return coefs[0] * x**2 + coefs[1] * x + coefs[2]

def find_coefficients(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    for a in range(-10, 11):
        for b in range(-10, 11):
            for c in range(-10, 11):
                coefs = np.array([a, b, c])
                if np.allclose(cuadratic_function(X, coefs), Y):
                    return coefs
    return np.array([])

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 12, 0])

coeficientes = find_coefficients(X, Y)
print("Ejercicio 8:")
print(f"Los coeficientes son {coeficientes}")

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [ ]:
def optimize_coefficients(X: np.ndarray, Y: np.ndarray, max_iterations: int = 10000) -> Tuple[np.ndarray, float]:
    coefs = np.random.randint(-10, 11, 3)
    learning_rate = 0.001
    best_error = float('inf')
    
    for _ in range(max_iterations):
        y_pred = cuadratic_function(X, coefs)
        error = np.sum((Y - y_pred)**2)
        
        if error < best_error:
            best_error = error
        
        if best_error <= 1:
            break
        
        new_coefs = coefs + np.random.uniform(-learning_rate, learning_rate, 3)
        new_y_pred = cuadratic_function(X, new_coefs)
        new_error = np.sum((Y - new_y_pred)**2)
        
        if new_error < error:
            coefs = new_coefs
    
    return coefs, best_error

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])

coeficientes, error_final = optimize_coefficients(X, Y)
print("\nEjercicio 9:")
print(f"Los coeficientes son {coeficientes} y el error es {error_final}")